In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

# Inputs
mu = expected_returns.return_model(prices, method='historical')  # Override TSLA manually
mu['TSLA'] = 0.21  # Our active view
S = risk_models.CovarianceShrinkage(prices).annualized_covariance  # Robust covariance

# Optimize
ef = EfficientFrontier(mu, S)

# Maximum Sharpe Ratio Portfolio (Tangency Portfolio)
sharpe_portfolio = ef.deepcopy()
weights_max_sharpe = sharpe_portfolio.max_sharpe(risk_free_rate=0.03)
ret_max_sharpe, vol_max_sharpe, _ = sharpe_portfolio.portfolio_performance(risk_free_rate=0.03)

# Minimum Volatility Portfolio
ef_min = EfficientFrontier(None, S)  # Don't use returns for min vol
weights_min_vol = ef_min.min_volatility()
ret_min_vol, vol_min_vol, _ = ef_min.portfolio_performance(risk_free_rate=0.03)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))
ef.plot_efficient_frontier(ax=ax, show_assets=False)

# Plot max Sharpe ratio
ax.scatter(vol_max_sharpe, ret_max_sharpe, marker='*', s=300, c='r', label='Max Sharpe Ratio')

# Plot min volatility
ax.scatter(vol_min_vol, ret_min_vol, marker='*', s=300, c='g', label='Min Volatility')

# Plot individual assets
for i, (label, color) in enumerate(zip(['TSLA', 'BND', 'SPY'], ['orange', 'blue', 'purple'])):
    ax.scatter(
        np.sqrt(S.loc[label, label]),
        mu[label],
        marker='o',
        s=100,
        c=color,
        label=label
    )

ax.set_title('Efficient Frontier with Optimal Portfolios')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()